# Custom 데이터로 CNN 모델 학습 실습
## 모듈 선언

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import torch.cuda as cuda
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder

## cuda(device), seed 선언

In [5]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(1)
if device=='cuda':
    torch.cuda.manual_seed_all(999)

cuda


## 데이터셋 불러오기

In [6]:
class custom_dataset(Dataset):
    def __init__(self, path, transfer):
        data = ImageFolder(root=path, transform=transfer)
        self.imgs = []
        self.labels = []

        for i, (img, label) in enumerate(data):
            self.imgs.append(img)
            self.labels.append([label])

        self.length = len(self.labels)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, item):
        data = self.imgs[item]
        target = torch.Tensor(self.labels[item]).to(torch.long).squeeze()
        return data, target


In [11]:

train_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])])

test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])])


train_dataset = custom_dataset('/content/drive/MyDrive/대학원/스터디/학부생스터디/6주차/custom_dataset/train/', train_transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True,)

test_dataset = custom_dataset('/content/drive/MyDrive/대학원/스터디/학부생스터디/6주차/custom_dataset/test/', test_transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=100,
                                           shuffle=True)

## 모델 선언

In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.imagesize = (224,224,3)

        self.conv1 = nn.Conv2d(3, 5, kernel_size=5, padding=2)      # 5,224,224
        self.conv2 = nn.Conv2d(5, 10, kernel_size=5, padding=2)     # 10,224,224
        self.conv3 = nn.Conv2d(10, 20, kernel_size=5, padding=2)    # 20,224,224
        self.maxpool1 = nn.MaxPool2d(2)                             # 20,112,112

        self.conv4 = nn.Conv2d(20, 20, kernel_size=3, padding=1)    # 20,112,112
        self.conv5 = nn.Conv2d(20, 20, kernel_size=3, padding=1)    # 20,112,112
        self.maxpool2 = nn.MaxPool2d(2)                             # 20,56,56

        self.fc1 = nn.Linear(20 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 4)


    def forward(self, x):
        in_size = x.size(0)

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.maxpool1(x))

        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.maxpool2(x))

        x = x.view(in_size, -1) 
        x = self.fc1(x)
        x = self.fc2(x)    

        return x

model = Net()
model = model.to(device)

## 학습 시작

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
Best = -1

for epoch in range(1, 11):
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    acc = 100. * correct / len(test_loader.dataset)
    if acc > Best:
        Best = acc
        
    print('Train Epoch {}: loss: {:.4f}'.format(epoch, loss))
    print('Test :  loss: {:.4f}, Accuracy: {}/{} {:.0f}% \n\n'.format(
        test_loss, correct, len(test_loader.dataset), acc))
print('Best_acc: ',Best.item())

Train Epoch 1: loss: 0.0667
Test :  loss: 0.6700, Accuracy: 87/100 87% 


Train Epoch 2: loss: 0.0075
Test :  loss: 0.6326, Accuracy: 84/100 84% 


Train Epoch 3: loss: 0.0041
Test :  loss: 0.7737, Accuracy: 80/100 80% 


Train Epoch 4: loss: 0.0022
Test :  loss: 0.8004, Accuracy: 84/100 84% 


Train Epoch 5: loss: 0.0025
Test :  loss: 0.8560, Accuracy: 83/100 83% 


Train Epoch 6: loss: 0.0025
Test :  loss: 0.8792, Accuracy: 82/100 82% 


Train Epoch 7: loss: 0.0008
Test :  loss: 0.9231, Accuracy: 80/100 80% 


Train Epoch 8: loss: 0.0007
Test :  loss: 0.9447, Accuracy: 80/100 80% 


Train Epoch 9: loss: 0.0009
Test :  loss: 0.9771, Accuracy: 80/100 80% 


Train Epoch 10: loss: 0.0009
Test :  loss: 0.9992, Accuracy: 81/100 81% 


Best_acc:  87.0
